# Webscraping, 3 ejercicios

Importamos las libreías

In [48]:
import requests
from bs4 import BeautifulSoup


# Ejercicio 1: Simple

Obtenemos datos sobre la torre Eiffel y los guardamos como archivo JSON

In [49]:
url = "https://www.toureiffel.paris/es/el-monumento/cifras-clave"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36',
}
response = requests.get(url, headers=headers)

if response.status_code == 200:
    print("Página recibida OK!")
    data = response.text
else:
    print("Error desde el servidor: " + response.status_code)


ConnectionError: HTTPSConnectionPool(host='www.toureiffel.paris', port=443): Max retries exceeded with url: /es/el-monumento/cifras-clave (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000028ABFE20760>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [ ]:
# Instancia del objeto beautiful soup con el código HTML
soup = BeautifulSoup(data, 'html.parser')


In [ ]:
#podemos ver todo el código de la página
soup.body

In [ ]:
# Podemos acceder a la tabla directamente desde el objeto soup
soup.table

In [ ]:
#Acceder a ka primera fila

table = soup.table
fila1 = table.tbody.find_all('tr')[1]
fila1

In [ ]:
# Acceder a los valores
print(fila1.th.text)
print(fila1.td.text)

In [ ]:
# Cantidad de Filas
filas = table.tbody.find_all('tr')
len(filas)

Iteramos las filas y creamos un diccionario de datos

In [ ]:
nuevo_dict = dict()
for fila in filas:
    clave = fila.th.text
    valor = fila.td.text
    nuevo_dict[clave] = valor
nuevo_dict

In [ ]:
# podemos hacer el mismo diccionario en una sóla línea con map y lambda
nuevo_dict = list(map(lambda a,b:{a.th.text:b.td.text},filas,filas))
nuevo_dict

## Guardamos los resultados

In [ ]:
import json

filename = "torre-eiffel.json"
with open(filename, 'w', encoding='utf-8') as f:
    json.dump(nuevo_dict, f, ensure_ascii=False, indent=4)


   # Ejercicio 2: Dificultad Media

Queremos extraer una tabla completa de la Wikipedia.<p>
Transformar datos y generar un nuevo Dataset.<p>
Guardamos en csv

In [ ]:
# importamos Pandas
import pandas as pd

In [ ]:
url = "https://es.wikipedia.org/wiki/Anexo:Monumentos_m%C3%A1s_visitados_del_mundo"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36',
}
response = requests.get(url, headers=headers)

if response.status_code == 200:
    print("Página recibida OK!")
    data = response.text
else:
    print("Error desde el servidor: " + response.status_code)


In [ ]:
# Vemos el código
data

In [ ]:
# Creamos el objeto BeautifulSoup
soup = BeautifulSoup(data, 'html.parser')

In [ ]:
# Vemos todas las tablas y sus clases css
print('Classes of each table:')
for table in soup.find_all('table'):
    print(table.get('class'))

In [ ]:
# Nos quedamos con la tabla indicada
table = soup.find('table', class_='sortable')

In [ ]:
# Veamos los valores de cabecera de la Tabla
cabecera = table.tbody.find_all('tr')[1]
cabecera

In [ ]:
cabecera = table.tbody.find_all('tr')[1]
tds = cabecera.find_all('td')
columnas = list(map(lambda a:a.text.strip(), tds))
columnas

In [ ]:
# Creamos un Dataframe vacio para almacenar los datos
df = pd.DataFrame()

In [ ]:
# Iteramos la tabla, transfornmamos y cargamos en el pandas

for row_num, row in enumerate(table.tbody.find_all('tr')):
    celdas = row.find_all('td')
    fila = {}
    if(row_num>1 and row_num<80 and celdas != []):
        for idx, celda in enumerate(celdas):
            if idx < len(columnas):
                if idx in [0,1,2,8]: #skip
                    pass 
                elif idx == 3: # Nombre
                    fila[columnas[idx]] = celda.a.text
                elif idx == 5: # Pais
                    enlace = celda.find_all("a")
                    try:
                        fila[columnas[idx]] = enlace[1].text
                    except:
                        fila[columnas[idx]] = celda.span.text
                elif idx == 6: #Visitantes
                    fila[columnas[idx]] = celda.text.strip().replace('\xa0','').replace('+','')
                else:
                    try:
                        fila[columnas[idx]] = celda.contents[0].text.strip()
                    except:
                        fila[columnas[idx]] = celda.text.strip()
        df = df.append(fila, ignore_index=True)

## Operamos con los datos

In [ ]:
# transformamos la columna de visitas a tipo numérico
df["Visitantes/año"] = df["Visitantes/año"].astype(int)
df.head(20)

In [ ]:
# Filtramos solo los monumentos de Francia
francia = df[df['País']=='Francia']
francia = francia[["Nombre","Visitantes/año"]]
francia = francia.set_index("Nombre")
francia

## Visualización de Datos

In [ ]:
from matplotlib import pyplot as plt
%matplotlib notebook

francia.sort_values("Visitantes/año").plot(kind="barh")

Vemos que la Torre Eiffel es rl 4to monumento de Francia con más visitas anuales

## Guardamos los datos en CSV

In [ ]:
df.to_csv("monumentos.csv")

# Ejercicio 3: Avanzado

Para este ejercicio utilizaremos Selenium.<p>
    Asegurate tener instalado el driver de <a href="https://chromedriver.chromium.org">Google Chrome</a> (también puede ser Fireforx, Safari o Edge)<p>
También de instalar el paquete con 'pip install selenium'<p>
Debes indicar la ruta del ejecutable de selenium (en Windows será chromedriver.exe)

In [54]:
CHROMIUM_PATH='C:\\Users\\drivers\\chromedriver_win32\\chromedriver.exe'

Definimos una función de espera, para simular la espera, como un humano

In [55]:
def wait_until(driver, secs):
    try:
        wait = WebDriverWait(driver, secs)
        wait.until(expected_conditions.element_to_be_clickable((By.XPATH, "//button[@id='btn1']")))
    except:
        pass

In [56]:
# Importamos las clases de Selenium
from selenium import webdriver
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

In [57]:
options = webdriver.ChromeOptions()
options.add_argument(
    '--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--ignore-ssl-errors')

driver = webdriver.Chrome(executable_path=CHROMIUM_PATH, chrome_options=options)

web = "https://www.instagram.com/"

driver.get(web)

C:\Users\carlo\AppData\Local\Temp\ipykernel_19812\3954340419.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=CHROMIUM_PATH, chrome_options=options)
C:\Users\carlo\AppData\Local\Temp\ipykernel_19812\3954340419.py:7: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=CHROMIUM_PATH, chrome_options=options)


## Aceptamos la Cookie

In [58]:
#cookie_button = driver.find_element_by_xpath(f"//button[text()='Aceptar todas']")
cookie_button = driver.find_element_by_xpath(f"//button[text()='Permitir cookies necesarias y opcionales']")
cookie_button.click()

AttributeError: 'WebDriver' object has no attribute 'find_element_by_xpath'

## Completamos el formulario de Login

In [59]:
# get element
element = driver.find_element_by_name("username")
# send keys
element.send_keys("TU_USUARIO")

wait_until(driver, 1)

element = driver.find_element_by_name("password")

element.send_keys("TU_PASSWORD")

AttributeError: 'WebDriver' object has no attribute 'find_element_by_name'

In [60]:
driver.find_element_by_id("loginForm").submit()

AttributeError: 'WebDriver' object has no attribute 'find_element_by_id'

## Ya estamos Logueados

In [ ]:
# Vamos a la página de Torre Eiffel
driver.get("https://www.instagram.com/explore/locations/307177342743013/torre-eiffel/")

In [36]:
# Filtramos los elementos de IMAGEN
elements = driver.find_elements_by_xpath('//img')
elements

[<selenium.webdriver.remote.webelement.WebElement (session="393c8c3df2c86e69c89a20b5cdda1678", element="8df7a14e-4f18-4e91-8034-bd17ff34cc36")>,
 <selenium.webdriver.remote.webelement.WebElement (session="393c8c3df2c86e69c89a20b5cdda1678", element="0a0fcd94-ca62-4421-be47-692571efc928")>,
 <selenium.webdriver.remote.webelement.WebElement (session="393c8c3df2c86e69c89a20b5cdda1678", element="cba810e3-cd31-4c31-a5c2-08b997a837b3")>,
 <selenium.webdriver.remote.webelement.WebElement (session="393c8c3df2c86e69c89a20b5cdda1678", element="22c8be14-c053-4955-90f1-974f1279e51a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="393c8c3df2c86e69c89a20b5cdda1678", element="47599c14-e443-4c19-8ae4-184c22a64723")>,
 <selenium.webdriver.remote.webelement.WebElement (session="393c8c3df2c86e69c89a20b5cdda1678", element="334b1175-471d-4d6b-92ca-dac48053f241")>,
 <selenium.webdriver.remote.webelement.WebElement (session="393c8c3df2c86e69c89a20b5cdda1678", element="351c7f18-5c41-424a-9d14-3c

In [37]:
len(elements)

60

## Guardamos las imagenes

Iteramos las imagenes, capturamos, hacemos scroll

In [38]:
directory="./screenshots" # el directorio de destino DEBE existir

px_scroll = 60

elements = driver.find_elements_by_xpath('//img')
for idx, element in enumerate(elements):
    with open(f"{directory}/capture-{idx}.png", 'wb') as file:
        try:
            file.write(element.screenshot_as_png)
        except:
            pass
    wait_until(driver, 1)

    driver.execute_script(f"window.scrollTo(0,{300+(idx*px_scroll)})")
    wait_until(driver, 1)

    if idx > 60:
        break


Finalizamos la sesion del navegador

In [39]:
driver.quit()